In [1]:
import geopy
import requests
import pandas as pd
from datetime import datetime
from geopy.distance import geodesic
import logging
import pytz
import threading
import time
import threading
import time
import os
from nbclient import NotebookClient
from nbformat import read
import sqlite3

# Função para criar a tabela no banco de dados SQLite (se ainda não existir)
def busca_pontos_linha(linha):
    url = f'https://transporteservico.urbs.curitiba.pr.gov.br/getPontosLinha.php?linha={linha:03}&c=821f0'
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        dados_json = response.json()

        # Verificar se os dados estão vazios
        if not dados_json:
            return None

        # Criar DataFrame a partir dos dados
        df = pd.DataFrame.from_records(dados_json)

        # Substituir vírgulas por pontos e converter tipos
        df['Latitude'] = df['LAT'].str.replace(',', '.').astype(float)
        df['Longitude'] = df['LON'].str.replace(',', '.').astype(float)
        df['SEQ'] = df['SEQ'].astype(int)
        df['ID_do_Itinerario'] = df['ITINERARY_ID'].astype(int)
        
        # Adicionar coluna COD
        df['COD'] = linha

        return df[['NOME', 'NUM', 'Latitude', 'Longitude', 'SEQ', 'GRUPO', 'SENTIDO', 'TIPO', 'ID_do_Itinerario', 'COD']]

    except requests.RequestException as e:
        pass
        #logging.error(f"Erro na requisição para a linha {linha}: {e}")
    except Exception as e:
        pass
        #logging.error(f"Erro ao processar a linha {linha}: {e}")
    return None

def criar_tabela():
    with sqlite3.connect('dados_processados.db') as conn:
        conn.execute('''
            CREATE TABLE IF NOT EXISTS onibus (
                COD TEXT,
                REFRESH TEXT,
                LAT_IN_TIME REAL,
                LON_IN_TIME REAL,
                CODIGOLINHA TEXT,
                ADAPT INTEGER,
                TIPO_VEIC INTEGER,
                TABELA INTEGER,
                SITUACAO TEXT,
                SITUACAO2 TEXT,
                SENT TEXT,
                TCOUNT INTEGER,
                SENTIDO_IN_TIME TEXT,
                HORA TEXT,
                FLAG_PROCES INTEGER,
                PARADA_MAIS_PROXIMA TEXT,
                DISTANCIA_MINIMA REAL,
                SEQ INTEGER,
                SEQ_MAX INTEGER,
                ITINERARIO INTEGER
            )
        ''')
        conn.commit()
def gravar_no_banco_pontos(df_result):
    try:
        with sqlite3.connect('dados_pontos.db') as conn:
            df_result.to_sql('onibus', conn, if_exists='append', index=False)
    except Exception as e:
        logging.error(f"Erro ao gravar no banco de dados: {e}")
def gravar_no_banco_processado(df_result):
    try:
        with sqlite3.connect('dados_processados.db') as conn:
            df_result.to_sql('onibus', conn, if_exists='append', index=False)
    except Exception as e:
        logging.error(f"Erro ao gravar no banco de dados: {e}")
def carregar_tabela_para_dataframe(nome_tabela, banco_de_dados):
    conn = sqlite3.connect(banco_de_dados)
    
    try:
        # Verificar se a tabela existe
        cursor = conn.cursor()
        cursor.execute("""
            SELECT name FROM sqlite_master WHERE type='table' AND name=?;
        """, (nome_tabela,))
        
        resultado = cursor.fetchone()

        if resultado:
            # Se a tabela existe, carregar seus dados para um DataFrame
            df = pd.read_sql_query(f"SELECT * FROM {nome_tabela}", conn)
            return df
        else:
            # Retornar um DataFrame vazio se a tabela não existir
            return pd.DataFrame()  
    finally:
        conn.close()  # Garantir que a conexão seja fechada

def reseta_index(df):
    df = df.reset_index(drop=True)
    return df

def calcular_distancia(lat1, lon1, lat2, lon2):
    try:
        if pd.isna(lat1) or pd.isna(lon1) or pd.isna(lat2) or pd.isna(lon2):
            raise ValueError("Coordenadas não podem ser NaN.")

        coords_1 = (lat1, lon1)
        coords_2 = (lat2, lon2)
        distancia = geodesic(coords_1, coords_2).meters
        return distancia
    except Exception as e:
        logging.error("Erro ao calcular distancia: %s", e)
        return None


def atribuir_parada(df_onibus, df_pontos):
    try:
        # Percorre todo o DataFrame df_onibus
        for i in range(df_onibus.shape[0]):
            # Obter as coordenadas da linha de ônibus atual
            lat_onibus = df_onibus['LAT_IN_TIME'].iloc[i]
            lon_onibus = df_onibus['LON_IN_TIME'].iloc[i]
            codigo_atual = df_onibus['CODIGOLINHA'].iloc[i]  # Código da linha atual
            itinerario_atual = df_onibus['ITINERARIO'].iloc[i]  # Itinerário da linha atual
            #print(lat_onibus, lon_onibus)
            df_pontos_itinerario = df_pontos[df_pontos['ID_do_Itinerario'] == itinerario_atual].reset_index(drop=True)
            if not df_pontos_itinerario.empty: 
                
                # Calcular distâncias para todas as paradas
                distancias = df_pontos_itinerario.apply(
                    lambda row: calcular_distancia(
                        lat_onibus,
                        lon_onibus,
                        row['Latitude'],
                        row['Longitude']
                    ),
                    axis=1
                )
    
                if not distancias.empty:  # Verifica se a série de distâncias não está vazia
                    index_parada_mais_proxima = distancias.idxmin()
                    distancia_minima = distancias.min()
                    nome_parada_mais_proxima = df_pontos_itinerario.loc[index_parada_mais_proxima, 'NOME']
                    mapeamento = dict(zip(df_pontos_itinerario['NOME'], df_pontos_itinerario['SEQ']))
                    
    
                    # Atualiza as informações diretamente no DataFrame df_onibus
                    df_onibus.at[i, 'PARADA_MAIS_PROXIMA'] = nome_parada_mais_proxima
                    df_onibus.at[i, 'SEQ'] = mapeamento[nome_parada_mais_proxima]
                    df_onibus.at[i, 'SEQ_MAX'] = df_pontos_itinerario['SEQ'].max()
                    df_onibus.at[i, 'DISTANCIA_MINIMA'] = distancia_minima
                else:
                    pass
    except Exception as e:
        logging.error("Erro ao atribuir paradas: %s", e)

    return df_onibus  # Retorna o DataFrame atualizado

def retorna_df_itinerario(df_onibus, df_pontos):
    try:
        if df_pontos.empty:
            logging.warning("O DataFrame df_pontos está vazio.")
            return None
        if df_onibus.empty:
            logging.warning("O DataFrame df_onibus está vazio.")
            return None

        if 'PARADA_MAIS_PROXIMA' not in df_onibus.columns:
            df_onibus['PARADA_MAIS_PROXIMA'] = 'inicializado'
        if 'DISTANCIA_MINIMA' not in df_onibus.columns:
            df_onibus['DISTANCIA_MINIMA'] = 0.0
        if 'SEQ' not in df_onibus.columns:
            df_onibus['SEQ'] = -1
        if 'SEQ_MAX' not in df_onibus.columns:
            df_onibus['SEQ_MAX'] = -1
        if 'ITINERARIO' not in df_onibus.columns:
            df_onibus['ITINERARIO'] = 0
        
        df_itinerario = carregar_tabela_para_dataframe('onibus','dados_itinerario.db')
        dicionario_itinerario = df_itinerario.set_index(['CODIGOLINHA', 'SENT'])['ITINERARIO'].to_dict()
        
        df_onibus['ITINERARIO'] = df_onibus.set_index(['CODIGOLINHA', 'SENT']).index.map(dicionario_itinerario)
        df_onibus = atribuir_parada(df_onibus, df_pontos)
        return df_onibus
    except Exception as e:
        logging.error("Erro ao retornar DataFrame do itinerário: %s", e)
        return None
        
def retorna_onibus():
    try:
        # Carregar o DataFrame de ônibus do banco de dados
        df_onibus = carregar_tabela_para_dataframe('onibus', 'dados_onibus.db')
        
        # Filtrar os ônibus com FLAG_PROCES igual a 0
        df_onibus = df_onibus[df_onibus['FLAG_PROCES'] == 0]
        df_onibus = df_onibus[df_onibus['SENT'] != '']
        # Resetar o índice
        df_onibus = df_onibus.reset_index(drop=True)

        # Atualizar o FLAG_PROCES para 1 indicando que os dados serão processados
        with sqlite3.connect('dados_onibus.db') as conn:
            conn.execute("UPDATE onibus SET FLAG_PROCES = 1 WHERE FLAG_PROCES = 0")
            conn.commit()

        return df_onibus
    except Exception as e:
        logging.error("Erro ao retornar DataFrame de ônibus: %s", e)
        return None


'''
# Criar a tabela se não existir
criar_tabela()



df_pontos = carregar_tabela_para_dataframe('onibus', 'dados_pontos.db')
df_onibus = retorna_onibus()
# Verificar se os DataFrames não estão vazios
if not df_onibus.empty and not df_pontos.empty:
    

    # Remover valores nulos
    df_onibus = df_onibus.dropna(subset=['LAT_IN_TIME', 'LON_IN_TIME'])
    df_pontos = df_pontos.dropna(subset=['Latitude', 'Longitude'])

    df = retorna_df_itinerario(df_onibus, df_pontos)

    # Salvar as alterações no banco
    gravar_no_banco_processado(df_onibus)
else:
    print("Erro: Um ou ambos os DataFrames estão vazios.")

'''

'\n# Criar a tabela se não existir\ncriar_tabela()\n\n\n\ndf_pontos = carregar_tabela_para_dataframe(\'onibus\', \'dados_pontos.db\')\ndf_onibus = retorna_onibus()\n# Verificar se os DataFrames não estão vazios\nif not df_onibus.empty and not df_pontos.empty:\n    \n\n    # Remover valores nulos\n    df_onibus = df_onibus.dropna(subset=[\'LAT_IN_TIME\', \'LON_IN_TIME\'])\n    df_pontos = df_pontos.dropna(subset=[\'Latitude\', \'Longitude\'])\n\n    df = retorna_df_itinerario(df_onibus, df_pontos)\n\n    # Salvar as alterações no banco\n    gravar_no_banco_processado(df_onibus)\nelse:\n    print("Erro: Um ou ambos os DataFrames estão vazios.")\n\n'

In [12]:
df = carregar_tabela_para_dataframe('onibus','dados_processados.db')

In [13]:
# Garantir que os valores nulos não causem problemas
df['SITUACAO'] = df['SITUACAO'].fillna('')
df['PARADA_MAIS_PROXIMA'] = df['PARADA_MAIS_PROXIMA'].fillna('inicializado')

# Filtrar o DataFrame
df = df[(df['SITUACAO'] != '') | (df['PARADA_MAIS_PROXIMA'] != 'inicializado')]

# Resetar o índice
df = df.reset_index(drop=True)


In [14]:
df

,COD,REFRESH,LAT_IN_TIME,LON_IN_TIME,CODIGOLINHA,ADAPT,TIPO_VEIC,TABELA,SITUACAO,SITUACAO2,SENT,TCOUNT,SENTIDO_IN_TIME,HORA,FLAG_PROCES,PARADA_MAIS_PROXIMA,DISTANCIA_MINIMA,SEQ,SEQ_MAX,ITINERARIO
0,BC010,16:23,-25.382566,-49.288068,171,1,1,1,NO HORÁRIO,REALIZANDO ROTA,VOLTA,1,234-BAIRRO PRIMAVERA (16:26),2024-10-23 16:23:28,0,"Rua Domingos Antônio Moro, 886 - Pilarzinho",26.311335,22,23,8655.0
1,GI852,16:23,-25.469910,-49.272365,662,1,7,1,NO HORÁRIO,REALIZANDO ROTA,IDA,1,672-PRACA RUI BARBOSA (16:41),2024-10-23 16:23:28,0,"Rua Alagoas, 745 - Guaíra",93.761353,14,29,13493.0
2,CI004,16:22,-25.439091,-49.236066,373,1,1,1,ATRASADO,REALIZANDO ROTA,VOLTA,1,806-TERMINAL BAIRRO ALTO (16:30),2024-10-23 16:23:28,0,"Av. Nossa Senhora da Penha, 266 - Cristo Rei",121.231378,16,33,15238.0
3,BI860,16:23,-25.430185,-49.271566,280,1,7,2,ADIANTADO,REALIZANDO ROTA,IDA,1,680-PRACA TIRADENTES (16:26),2024-10-23 16:23:28,0,"Rua Barão do Serro Azul, 217 - São Francisco",281.124874,35,35,609.0
4,BI893,16:21,-25.373775,-49.235210,280,1,7,1,NO HORÁRIO,REALIZANDO ROTA,VOLTA,1,190-BAIRRO NSA.SRA.DE NAZARE (16:27),2024-10-23 16:23:28,0,"Rua José Antônio Leprevost, 974 - Santa Candida",15.381375,30,32,608.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82605,HL495,13:13,-25.435136,-49.273281,707,1,3,1,NO HORÁRIO,REALIZANDO ROTA,IDA,1,672-PRACA RUI BARBOSA (13:16),2024-10-26 13:13:47,0,Praça Rui Barbosa - 707 Tatuquara / Centro,17.519772,5,5,15115.0
82606,HL494,13:13,-25.456541,-49.250746,707,1,3,2,NO HORÁRIO,REALIZANDO ROTA,IDA,1,672-PRACA RUI BARBOSA (13:34),2024-10-26 13:13:47,0,Estação Tubo PUC,397.625065,3,5,15115.0
82607,HL498,13:13,-25.525633,-49.297061,707,1,3,5,ATRASADO,REALIZANDO ROTA,VOLTA,1,1690-TERMINAL TATUQUARA (13:34),2024-10-26 13:13:47,0,Estação Xaxim,3834.115065,4,5,15116.0
82608,JL320,13:13,-25.525990,-49.297221,707,1,3,4,ATRASADO,REALIZANDO ROTA,VOLTA,1,1690-TERMINAL TATUQUARA (13:16),2024-10-26 13:13:47,0,Estação Xaxim,3876.014329,4,5,15116.0


In [15]:
df = df.head(10)

In [16]:
df

,COD,REFRESH,LAT_IN_TIME,LON_IN_TIME,CODIGOLINHA,ADAPT,TIPO_VEIC,TABELA,SITUACAO,SITUACAO2,SENT,TCOUNT,SENTIDO_IN_TIME,HORA,FLAG_PROCES,PARADA_MAIS_PROXIMA,DISTANCIA_MINIMA,SEQ,SEQ_MAX,ITINERARIO
0,BC010,16:23,-25.382566,-49.288068,171,1,1,1,NO HORÁRIO,REALIZANDO ROTA,VOLTA,1,234-BAIRRO PRIMAVERA (16:26),2024-10-23 16:23:28,0,"Rua Domingos Antônio Moro, 886 - Pilarzinho",26.311335,22,23,8655.0
1,GI852,16:23,-25.469910,-49.272365,662,1,7,1,NO HORÁRIO,REALIZANDO ROTA,IDA,1,672-PRACA RUI BARBOSA (16:41),2024-10-23 16:23:28,0,"Rua Alagoas, 745 - Guaíra",93.761353,14,29,13493.0
2,CI004,16:22,-25.439091,-49.236066,373,1,1,1,ATRASADO,REALIZANDO ROTA,VOLTA,1,806-TERMINAL BAIRRO ALTO (16:30),2024-10-23 16:23:28,0,"Av. Nossa Senhora da Penha, 266 - Cristo Rei",121.231378,16,33,15238.0
3,BI860,16:23,-25.430185,-49.271566,280,1,7,2,ADIANTADO,REALIZANDO ROTA,IDA,1,680-PRACA TIRADENTES (16:26),2024-10-23 16:23:28,0,"Rua Barão do Serro Azul, 217 - São Francisco",281.124874,35,35,609.0
4,BI893,16:21,-25.373775,-49.235210,280,1,7,1,NO HORÁRIO,REALIZANDO ROTA,VOLTA,1,190-BAIRRO NSA.SRA.DE NAZARE (16:27),2024-10-23 16:23:28,0,"Rua José Antônio Leprevost, 974 - Santa Candida",15.381375,30,32,608.0
5,DC854,16:21,-25.470068,-49.207776,331,1,7,1,NO HORÁRIO,REALIZANDO ROTA,CIRCULAR,1,837-TERMINAL CENTENARIO (16:40),2024-10-23 16:23:28,0,Terminal Centenário - 331 - Mercúrio,107.241468,14,14,11276.0
6,BI019,16:22,-25.390495,-49.245360,225,1,1,1,NO HORÁRIO,REALIZANDO ROTA,VOLTA,1,810-TERMINAL BOA VISTA (16:25),2024-10-23 16:23:28,0,"Rua Nossa Senhora de Nazaré, 1211 - Boa Vista",62.973944,13,15,14933.0
7,BI876,16:23,-25.392123,-49.225713,213,1,7,2,NO HORÁRIO,REALIZANDO ROTA,IDA,1,268-BAIRRO SAO JOAO (16:30),2024-10-23 16:23:28,0,"Av. Prefeito Erasto Gaertner, 2419 - Bacacheri",83.315242,7,18,12613.0
8,BI881,16:23,-25.435215,-49.275635,385,1,7,1,NO HORÁRIO,REALIZANDO ROTA,IDA,1,94-BAIRRO CRISTO REI (16:45),2024-10-23 16:23:28,0,"R. Visc. de Nacar, 1210 - Centro",332.602120,19,19,674.0
9,BC304,16:23,-25.368205,-49.259538,245,1,3,2-2,NO HORÁRIO,REALIZANDO ROTA,VOLTA,1,1539-BAIRRO ANITA GARIBALDI (16:32),2024-10-23 16:23:28,0,inicializado,0.000000,-1,-1,NaN


In [17]:
import pandas as pd
from geopy.geocoders import Nominatim
import time
import requests_cache

# Configurar o cache para evitar chamadas repetidas à API
requests_cache.install_cache('geopy_cache', expire_after=3600)  # Cache expira após 1 hora

# Inicializar o geolocalizador
geolocator = Nominatim(user_agent="MeuProjetoGeolocalizacao")

# Função para obter informações do endereço
def obter_endereco_detalhado(lat, lon):
    try:
        time.sleep(1)  # Espera para não sobrecarregar a API
        location = geolocator.reverse((lat, lon), language='pt', exactly_one=True)
        if location:
            address = location.raw.get('address', {})
            return {
                "cep": address.get("postcode", None),
                "nome_rua": address.get("road", None),
                "nome_bairro": address.get("suburb", None)
            }
        return {"cep": None, "nome_rua": None, "nome_bairro": None}
    except Exception as e:
        print(f"Erro ao obter endereço para {lat}, {lon}: {e}")
        return {"cep": None, "nome_rua": None, "nome_bairro": None}



# Aplicar a função de geocodificação
enderecos = df.apply(
    lambda row: obter_endereco_detalhado(row['LAT_IN_TIME'], row['LON_IN_TIME']),
    axis=1
)

# Expandir os resultados em colunas separadas
enderecos_df = pd.DataFrame(enderecos.tolist())
df = pd.concat([df, enderecos_df], axis=1)




     COD REFRESH  LAT_IN_TIME  LON_IN_TIME CODIGOLINHA  ADAPT  TIPO_VEIC  \
0  BC010   16:23   -25.382566   -49.288068         171      1          1   
1  GI852   16:23   -25.469910   -49.272365         662      1          7   
2  CI004   16:22   -25.439091   -49.236066         373      1          1   
3  BI860   16:23   -25.430185   -49.271566         280      1          7   
4  BI893   16:21   -25.373775   -49.235210         280      1          7   
5  DC854   16:21   -25.470068   -49.207776         331      1          7   
6  BI019   16:22   -25.390495   -49.245360         225      1          1   
7  BI876   16:23   -25.392123   -49.225713         213      1          7   
8  BI881   16:23   -25.435215   -49.275635         385      1          7   
9  BC304   16:23   -25.368205   -49.259538         245      1          3   

  TABELA    SITUACAO        SITUACAO2  ...                 HORA  FLAG_PROCES  \
0      1  NO HORÁRIO  REALIZANDO ROTA  ...  2024-10-23 16:23:28            0   
1  

In [11]:
df

,COD,REFRESH,LAT_IN_TIME,LON_IN_TIME,CODIGOLINHA,ADAPT,TIPO_VEIC,TABELA,SITUACAO,SITUACAO2,...,DISTANCIA_MINIMA,SEQ,SEQ_MAX,ITINERARIO,cep,nome_rua,nome_bairro,cep,nome_rua,nome_bairro
0,BC010,16:23,-25.382566,-49.288068,171,1,1,1,NO HORÁRIO,REALIZANDO ROTA,...,26.311335,22,23,8655.0,82110-350,Rua Domingos Antônio Moro,Pilarzinho,82110-350,Rua Domingos Antônio Moro,Pilarzinho
1,GI852,16:23,-25.469910,-49.272365,662,1,7,1,NO HORÁRIO,REALIZANDO ROTA,...,93.761353,14,29,13493.0,80630-150,Rua Alagoas,Guaíra,80630-150,Rua Alagoas,Guaíra
2,CI004,16:22,-25.439091,-49.236066,373,1,1,1,ATRASADO,REALIZANDO ROTA,...,121.231378,16,33,15238.0,80050-520,Rua Urbano Lopes,Cristo Rei,80050-520,Rua Urbano Lopes,Cristo Rei
3,BI860,16:23,-25.430185,-49.271566,280,1,7,2,ADIANTADO,REALIZANDO ROTA,...,281.124874,35,35,609.0,80020-060,Praça Tiradentes,Centro,80020-060,Praça Tiradentes,Centro
4,BI893,16:21,-25.373775,-49.235210,280,1,7,1,NO HORÁRIO,REALIZANDO ROTA,...,15.381375,30,32,608.0,82640-040,Rua José Antônio Leprevost,Santa Cândida,82640-040,Rua José Antônio Leprevost,Santa Cândida
5,DC854,16:21,-25.470068,-49.207776,331,1,7,1,NO HORÁRIO,REALIZANDO ROTA,...,107.241468,14,14,11276.0,82960-180,Rua Engenheiro Benedito Mário da Silva,Cajuru,82960-180,Rua Engenheiro Benedito Mário da Silva,Cajuru
6,BI019,16:22,-25.390495,-49.245360,225,1,1,1,NO HORÁRIO,REALIZANDO ROTA,...,62.973944,13,15,14933.0,82560-040,Rua João Havro,Boa Vista,82560-040,Rua João Havro,Boa Vista
7,BI876,16:23,-25.392123,-49.225713,213,1,7,2,NO HORÁRIO,REALIZANDO ROTA,...,83.315242,7,18,12613.0,82600-360,Rua Achilles Stenghel,Bacacheri,82600-360,Rua Achilles Stenghel,Bacacheri
8,BI881,16:23,-25.435215,-49.275635,385,1,7,1,NO HORÁRIO,REALIZANDO ROTA,...,332.602120,19,19,674.0,80010-030,Rua Visconde de Nacar,Centro,80010-030,Rua Visconde de Nacar,Centro
9,BC304,16:23,-25.368205,-49.259538,245,1,3,2-2,NO HORÁRIO,REALIZANDO ROTA,...,0.000000,-1,-1,NaN,82700-110,Avenida Anita Garibaldi,Barreirinha,82700-110,Avenida Anita Garibaldi,Barreirinha


In [9]:
df_pontos = carregar_tabela_para_dataframe('onibus', 'dados_pontos.db')
df_pontos

,NOME,NUM,Latitude,Longitude,SEQ,GRUPO,SENTIDO,TIPO,ID_do_Itinerario,COD
0,"Av. Manoel Ribas, 531 - Mercês",110037,-25.422483,-49.284328,37,,Horário,Novo mobiliário,481,10
1,"Rua Augusto Stresser, 517 - Juvevê",130238,-25.417083,-49.256173,6,,Horário,Novo mobiliário,481,10
2,"Rua Chile, 821 - Rebouças",140051,-25.447454,-49.257707,18,,Horário,Novo mobiliário,481,10
3,"R. Dep. Mário de Barros, 1681 - Centro Cívico",120112,-25.411726,-49.270902,42,,Horário,Novo mobiliário,481,10
4,"R. Alm. Tamandaré, 272 - Alto da Rua Quinze",130268,-25.430345,-49.255576,10,,Horário,Novo mobiliário,481,10
...,...,...,...,...,...,...,...,...,...,...
14297,"Rua João Golin, 2 - Butiatuvinha",190753,-25.393788,-49.344422,10,,Ouro Verde,Placa em poste,8120,948
14298,"Av. Napoleão Manosso, 553 - Butiatuvinha",190155,-25.393658,-49.341903,6,,Terminal Santa Felicidade,Chapéu chinês,8136,948
14299,"Rua Via Veneto, 1593 - Santa Felicidade",190239,-25.399836,-49.333621,25,,Terminal Santa Felicidade,Novo mobiliário,12854,948
14300,"Rua Professor Francisco Zardo, 479 - Santa Fel...",190167,-25.404782,-49.340524,21,,Terminal Santa Felicidade,Chapéu chinês,12854,948
